# Esame del 21 Luglio 2025
### Francesco Cusenza - 0780292

In [ ]:
import numpy as np
from numpy.linalg import eig 

# Data la catena di markov di ordine 1 determinare 
# la matrice stocastica associata, qual'è il rate di 
# convergenza verso la soluzione stazionaria e qual è 
# la matrice che descrive la distribuzione di probabilità

#ipotesi di matrice stocastica
# P = np.array([[ 0.9, 0.1], #70% di 1 in 1, 30 di 1 in 2, stato di partenza riga, arrivo colonna
#               [0.1, 0.9]]) #diag = prob di restare fermo. Sumrow = 1

In [2]:
# Se avessimo un file .fasta potremmo comportarci così
from Bio.SeqIO import read
import numpy as np

record = read("Genome.fasta","fasta")
x_r = np.array(list(record.seq)) # Converti in array di lettere
x = np.zeros(len(x_r),dtype=int) # Codifica sequenza in numeri

for i in range(len(x)):
    if x_r[i] == 'A':
        x[i] = 0
    elif x_r[i] == 'C':
        x[i] = 1
    elif x_r[i] == 'G':
        x[i] = 2
    else:
        x[i] = 3

n = 4
M = np.zeros((n,n))

In [3]:
x = np.loadtxt("markov_chain.dat")
print(f'CATENA DI MARKOV:\n{x}')

states = np.array(x[:,1], dtype=int)
states = states - 1 # Cosi' partono da 0
N = 3 # Numero stati

CATENA DI MARKOV:
[[   0.    1.]
 [   1.    2.]
 [   2.    1.]
 ...
 [ 998.    1.]
 [ 999.    1.]
 [1000.    2.]]


In [ ]:
# Stima matrice di transizione
trans = np.zeros((N, N), dtype=int)

# ipotesi di matrice stocastica
# P = np.array([[ 0.8, 0.2], 
#               [0.1, 0.9]]) 

for t in range(len(states)-1):
    i = states[t]
    j = states[t+1]
    trans[i,j] += 1

P = trans / trans.sum(axis=1, keepdims=True) # Normalizzazione di ogni riga
# np.sum con axis=1 somma ogni riga, senza somma tutti gli el
print(f'MATRICE DI TRANSIZIONE:\n{P}')

MATRICE DI TRANSIZIONE:
[[0.506      0.242      0.252     ]
 [0.53846154 0.10121457 0.36032389]
 [0.44664032 0.40316206 0.15019763]]


In [5]:
# Calcolo di autovalori e auovettori
eigvals, eigvec = np.linalg.eig(P)
print(f'AUTOVALORI:\n{eigvals}\nAUTOVETTORI (colonne):\n{eigvec}')

AUTOVALORI:
[ 1.          0.01226365 -0.25485145]
AUTOVETTORI (colonne):
[[ 0.57735027  0.558766   -0.01282106]
 [ 0.57735027 -0.3640941  -0.70159328]
 [ 0.57735027 -0.74512821  0.71246228]]


In [7]:
sort_eigvals = np.sort(np.abs(eigvals))[::-1] # Mettono in ordine di modulo decrescente gli autovalori
print(f'AUTOVALORI ORDINATI:\n{sort_eigvals}') # rate = sort_eigvals[1]
print(f'Il secondo autovalore più grande determina la convergenza alla stazionarietà') #il primo è sempre circa 1

AUTOVALORI ORDINATI:
[1.         0.25485145 0.01226365]
Il secondo autovalore più grande determina la convergenza alla stazionarietà


In [8]:
# Determinazione stato stazionario
eigvalsT, eigvecT = np.linalg.eig(P.T) # Autovalori e autovettori della matrice trasposta, poichè pi_staz.T = P.T x pi_staz.T
stationarity = eigvecT[:, np.isclose(eigvalsT, 1)].real # Prendo la parte reale dell'autovettore associato ad 1
stationarity /= np.sum(stationarity) # Normalizzo l'autovettore associato ad 1
print(f'STAZIONARIETA\':\n{stationarity}') #a passi inf gli stati sarenno suddivisi nell'autovettore associato ad autovalore 1, le cui entries sommano a 1. se la mattrans è simmetrica, le entries sono equiprobabili
print(f'L \'autovettore associato al primo autovalore (1) corrisponde alla distribuzione di probabilità in cui convergerà il processo stocastico\nSi nota che al passo t->inf gli stati non sono equiprobabili, poichè in la matrice di transizione è asimetrica, in particolare favorisce lo stato 1')

STAZIONARIETA':
[[0.49902367]
 [0.24788894]
 [0.2530874 ]]
L 'autovettore associato al primo autovalore (1) corrisponde alla distribuzione di probabilità in cui convergerà il processo stocastico
Si nota che al passo t->inf gli stati non sono equiprobabili, poichè in la matrice di transizione è asimetrica, in particolare favorisce lo stato 1


In [7]:
# Controllo sul vettore stazionarietà, deve valere anche pi_staz = pi_staz x P
print("Check pi_staz = pi_staz x P:", np.allclose(stationarity.T @ P, stationarity.T))
print("Check pi_staz.T = P.T x pi_staz.T:", np.allclose(P.T @ stationarity, stationarity))

Check pi_staz = pi_staz x P: True
Check pi_staz.T = P.T x pi_staz.T: True
